# SVM Image Classifier

(Warning: This instance needs at least 25GB of ram)

In [0]:
# Importing libraries
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import pandas as pd

from skimage.color import rgb2grey
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import data, exposure

import time


# Loading the Dataset

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!unzip "/content/gdrive/My Drive/EE499/Project/Dataset.zip" -d "/Data"

Streaming output truncated to the last 5000 lines.
  inflating: /Data/499 Project/sea/15615.jpg  
  inflating: /Data/499 Project/sea/15617.jpg  
  inflating: /Data/499 Project/sea/15618.jpg  
  inflating: /Data/499 Project/sea/15622.jpg  
  inflating: /Data/499 Project/sea/15635.jpg  
  inflating: /Data/499 Project/sea/15637.jpg  
  inflating: /Data/499 Project/sea/15651.jpg  
  inflating: /Data/499 Project/sea/15656.jpg  
  inflating: /Data/499 Project/sea/15659.jpg  
  inflating: /Data/499 Project/sea/15661.jpg  
  inflating: /Data/499 Project/sea/15666.jpg  
  inflating: /Data/499 Project/sea/15671.jpg  
  inflating: /Data/499 Project/sea/15691.jpg  
  inflating: /Data/499 Project/sea/1570.jpg  
  inflating: /Data/499 Project/sea/15703.jpg  
  inflating: /Data/499 Project/sea/15711.jpg  
  inflating: /Data/499 Project/sea/15715.jpg  
  inflating: /Data/499 Project/sea/15736.jpg  
  inflating: /Data/499 Project/sea/15753.jpg  
  inflating: /Data/499 Project/sea/15769.jpg  
  inflatin

In [0]:
def load_image_files(container_path, dimension=(150, 150)):
    """
    Load image files with categories as subfolder names 
    which performs like scikit-learn sample dataset
    
    Parameters
    ----------
    container_path : string or unicode
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to
        
    Returns
    -------
    Bunch
    """
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]
    count = 0
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file)
            if np.shape(img) != (150,150,3):
              count += 1
            else:
              #img_resized = resize(img, dimension, anti_aliasing=False, mode='reflect')
              #flat_data.append(img_resized.flatten()) 
              images.append(img)
              target.append(i)
    #flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)
    print("Total images omitted: ",count)
    return Bunch(# data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images)

In [5]:
image_dataset = load_image_files ("/Data/499 Project")

Total images omitted:  55


In [0]:
plt.figure(figsize=(12,10))
for i in range(6):
  plt.subplot(3,3,i+1)
  idx = list(image_dataset.target).index(i)
  plt.imshow(image_dataset['images'][idx])
  label = image_dataset['target'][idx]
  plt.title(image_dataset['target_names'][label])

plt.show()

# Features 
Unlike CNNs where it inherently has feature extraction capabilites, we need to provide it features in one dimension.

Histogram of Oriented Gradients may help with object detection.
https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html

In [0]:
# Visualizing HOG featuers
for i in range(6):
  idx = list(image_dataset.target).index(i)
  image = image_dataset['images'][idx]
# We convert the image to greyscale to avoid giving priority to the dominant color
  image = rgb2grey(image)
# Using Histogram of Oriented Gradients
  fd, hog_image = hog(image, orientations=9, pixels_per_cell=(16, 16),
                      visualize=True, multichannel=False)
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

  ax1.axis('off')
  ax1.imshow(image, cmap=plt.cm.gray)
  ax1.set_title('Input image')

  # Rescale histogram for better display
  hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

  ax2.axis('off')
  ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
  ax2.set_title('Histogram of Oriented Gradients')
  plt.show()


In [0]:
def create_features(img):
    # flatten three channel color image
    color_features = img.flatten()
    # convert image to greyscale
    grey_image = rgb2grey(img)
    # get HOG features from greyscale image
    hog_features = hog(grey_image, orientations=9, pixels_per_cell=(16, 16),
                      visualize=False, multichannel=False)
    # combine color and hog features into a single array
    flat_features = np.hstack((color_features,hog_features))
    return flat_features


In [0]:
def create_feature_matrix(image_dataset):
  features_list = []
  for image in image_dataset.images:
    image_features = create_features(image)
    features_list.append(image_features)
  feature_matrix = np.array(features_list)
  return feature_matrix
feature_matrix = create_feature_matrix(image_dataset)

# Standardization

Princple Component Analysis did not work due to RAM limitations. (Might work with batches)


In [0]:
scaler = StandardScaler()
feature_matrix = scaler.fit_transform(feature_matrix)

In [0]:
# pca = PCA(n_components=150)

# feat_pca = pca.fit_transform(feature_matrix)

# print('PCA Features', np.shape(feat_pca))

In [0]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(
    feature_matrix, image_dataset.target, test_size=0.5,random_state=42)

# Training 

In [14]:
# Training the model
start = time.time()
model = svm.SVC(C = 1, kernel = 'rbf', max_iter=150, random_state=42)
model.fit(X_train, y_train)
end = time.time()

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=150).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [8]:
# Time needed to perform 150 iterations (in minutes)
print("Time taken during training: {:.2f} minutes ".format((end - start)/60))

Time taken during training: 1.22 minutes 


In [0]:
y_pred = model.predict(X_test)

In [0]:
# 50 Iterations
accuracy_score(y_test,y_pred)

0.5861517751977116

In [0]:
# 150 Iterations
accuracy_score(y_test,y_pred)

0.6539458186101296

In [0]:
# 150 Iterations confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
# Using sklearn's confusion matrix plot
def plot_confusion_matrix(labels, pred_labels, classes):
    
    fig = plt.figure(figsize = (7, 7));
    plt.title("Confusion Matrix (Multi-class)")
    ax = fig.add_subplot(1, 1, 1);
    cm = confusion_matrix(labels, pred_labels);
    cm = ConfusionMatrixDisplay(cm, classes);
    cm.plot(values_format = 'd', cmap = 'Blues', ax = ax)
    plt.xticks(rotation = 20)
    plt.show()
plot_confusion_matrix(y_test, y_pred, image_dataset.target_names)
print("The accuracy of the model is:  {:.2f}%".format(100*accuracy_score(y_test,y_pred)))

# References 

[1] For loading the dataset: https://github.com/whimian/SVM-Image-Classification/blob/master/

[2] For feature extraction: https://rpubs.com/Sharon_1684/454441

[3] Scikit-image HOG documentation: https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html